# <center>Record Linkage</center>
## <center>Vince Marinelli<br>CPSC548 Spring 2025</center>

Record Linkage refers to the task of linking records using a matching set of attributes shared by the two records. For example, we often need to match a list customers with a list of payees, or a list of patients with a list of users of a specific medication, etc. Another common example is the identification and removal of duplicate entries within a dataset. Record Linkage has also been referred to as data linkage, entity resolution, and data matching (<ins>Christen. 2019</ins>).

In the simplest case, we have a shared attribute that represents a unique identifier for the record (e.g. SSN, TIN, ISBN, etc). In this case we may assume that this identifier absolutely and completely identities the unique entity. Unfortunately, outside of the world of controlled schemas this is rarely true. More often, we need to use the full set of attributes that can be mapped between the two rows to develop a level of confidence that the two rows are the same. The attributes used to match are referred to as Quasi-Identifiers (QIDs). Each QID contributes a specific weight to the overall match probability that is proportional to the amount of information that the QID contributes. For example, a matching street address contributes a larger weight than a matching marital status because the street address has a higher cardinality / lower match probability then marital status.

The seminal work on record matching, **A Theory for Record Linkage**. was published in 1969 by Fellegi and Sunter. In the paper, the authors lay out a statistical theoretical basis for optimal record matching. It was later shown that this statistical basis is effectively the same as a Naive Baisian Classifer with the conditional independence assumption relaxed (<ins>Winkler, 2012</ins>). These statistical models remain mainstream today but are being challenged by newer models that use other types of classifiers.

In his 2012 book **Data Matching**, Peter Christen lays out the basic process that has historically been followed. The process contains the following steps (<ins>Christen, 2012, pp. 24-35</ins>):

1. Preprocessing - scrub and format record sets to be compared
2. Indexing - match each record from one data to records in the comparison data set
3. Comparison - compute similarity between a record and all possible comparators
4. Classification - classify records as matches or non-matches (or possible matches in some cases)
5. Evaluation - review the performance of the classification

Before delving into these steps in more detail, we will first review the Python library we will be using both for test data and for the execution of many of the above steps.

### Using the Python Record Linkage Toolkit (PRLT)
The [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/index.html) is a PyPi-hosted library that implements a standard set of record linkage activities. It includes tools for preprocessing data, indexing, comparison, classification and evaluation. The toolkit also contains several datasets that can be used to train and test algorithms.

First, we'll import all the modules that we'll be using in this workbook, including PRLT.

In [115]:
import numpy as np
import pandas as pd
import random
import re
import recordlinkage

from datasketch import MinHash, MinHashLSH
from pprint import pprint
from recordlinkage.datasets import load_febrl4
from recordlinkage.base import BaseIndexAlgorithm
from textnoisr import noise

Now that the modules have been imported, we will load data from the Freely Extensible Biomedical Record Linkage (febrl) package which is included in PRLT. Calling the `load_febrl4` method returns two datasets - one which contains all original records and a second that contains copies of the records from the first dataset, with duplicates included. A third output, a Pandas MultiIndex, can optionally be returned that contains the actual map from rows in the first dataframe to rows in the second (<ins>de Bruin, 2023</ins>). We'll retrieve all three objects and display them.

In [116]:
dfA, dfB, miTrueLinks = load_febrl4(return_links=True)
dfA

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688
...,...,...,...,...,...,...,...,...,...,...
rec-2153-org,annabel,grierson,97,mclachlan crescent,lantana lodge,broome,2480,nsw,19840224,7676186
rec-1604-org,sienna,musolino,22,smeaton circuit,pangani,mckinnon,2700,nsw,19890525,4971506
rec-1003-org,bradley,matthews,2,jondol place,horseshoe ck,jacobs well,7018,sa,19481122,8927667


In [117]:
print(f'Length of dataframe dfA: {len(dfA)}')

Length of dataframe dfA: 5000


In [118]:
dfB

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884
...,...,...,...,...,...,...,...,...,...,...
rec-4495-dup-0,connor,belperio,15,NaN,NaN,ryde,2570,nsw,19170518,5394641
rec-4211-dup-0,daniel,maspn,9,derrington crescent,el pedro caravan park,sunnybank,4350,vic,19500705,5525378
rec-3131-dup-0,samuel,crofs,613,banjine street,kurrajong vlge,pengzin,2230,qld,19410531,4467228


In [119]:
print(f'Length of dataframe dfB: {len(dfB)}')

Length of dataframe dfB: 5000


In [120]:
miTrueLinks

MultiIndex([(   'rec-0-org',    'rec-0-dup-0'),
            (   'rec-1-org',    'rec-1-dup-0'),
            (   'rec-2-org',    'rec-2-dup-0'),
            (   'rec-3-org',    'rec-3-dup-0'),
            (   'rec-4-org',    'rec-4-dup-0'),
            (   'rec-5-org',    'rec-5-dup-0'),
            (   'rec-6-org',    'rec-6-dup-0'),
            (   'rec-7-org',    'rec-7-dup-0'),
            (   'rec-8-org',    'rec-8-dup-0'),
            (   'rec-9-org',    'rec-9-dup-0'),
            ...
            ('rec-4990-org', 'rec-4990-dup-0'),
            ('rec-4991-org', 'rec-4991-dup-0'),
            ('rec-4992-org', 'rec-4992-dup-0'),
            ('rec-4993-org', 'rec-4993-dup-0'),
            ('rec-4994-org', 'rec-4994-dup-0'),
            ('rec-4995-org', 'rec-4995-dup-0'),
            ('rec-4996-org', 'rec-4996-dup-0'),
            ('rec-4997-org', 'rec-4997-dup-0'),
            ('rec-4998-org', 'rec-4998-dup-0'),
            ('rec-4999-org', 'rec-4999-dup-0')],
           length=5000)

### Preprocessing
When using this test dataset, we only need to do some minor preprocessing because the column names and formats all match. In real-world scenarios, we would likely need to map names, addresses, phone numbers, etc. to the same formats, convert data types, etc. There are some data scrubbing steps we will take here such as correcting nonsensical data (e.g. the word "NULL" in cells) and properly formatting data (e.g. express phone numbers as strings, convert birth dates to strongly-typed datetime, etc).

In this case, we also add some noise to the surname column in order to more effectively exercise the later steps in the process. To do this, we use a library called [textnoizr](https://github.com/preligens-lab/textnoisr) which randomly permutes strings by inserting, deleting, substituting or swapping letters. We apply this function to about 10% of the entries in the `surname` column.

In [121]:
# replace NaNs with empty strings
dfA = dfA.map(lambda x: '' if pd.isna(x) else x)
dfB = dfB.map(lambda x: '' if pd.isna(x) else x)

# cast date_of_birth as datetime64 in a new column
dfA['dob_typed'] = pd.to_datetime(dfA['date_of_birth'],format='%Y%m%d',errors='coerce')
dfB['dob_typed'] = pd.to_datetime(dfB['date_of_birth'],format='%Y%m%d',errors='coerce')

# create a noise function that adds ~10% noise, ~10% of the time
def add_noise(input):
    p = random.random()
    noise_generator = noise.CharNoiseAugmenter(noise_level=0.1)
    if p > 0.9:
        output = noise_generator.add_noise(input)
    else:
        output = input
    return output
# add noise to the surname field
# dfB['surname'] = dfB['surname'].map(lambda x: add_noise(x))
# dfB

### Indexing
The next step in the process is to link each row in the source to each row in the target so that we can check for matches based on weighted scores. In it's simplest form, this results in a full cartesian product of `dfA X dfB`. Clearly this does not scale well so most often a technique called Blocking is applied in which obvious non-matches are not included in the indexes. Simpler techniques block based on exact matches on one or more attributes (columns) while more complex and interesting techniques relax the matching criteria using techniques such as clustering, fuzzy hashing, and bloom filter grouping.

The PRLT library includes fairly simple Indexing methods that include Full (no Blocking), Block which uses exact matches on specified attributes, and SortedNeighborhood which sorts data from both datasets together and groups data from two different sets according a window size (<ins>de Bruin, 2023</ins>). Interestingly, this concept is similar in functionality to convolution.

In [122]:
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)
pairs

MultiIndex([('rec-1070-org',  'rec-561-dup-0'),
            ('rec-1070-org', 'rec-2642-dup-0'),
            ('rec-1070-org',  'rec-608-dup-0'),
            ('rec-1070-org', 'rec-3239-dup-0'),
            ('rec-1070-org', 'rec-2886-dup-0'),
            ('rec-1070-org', 'rec-4285-dup-0'),
            ('rec-1070-org',  'rec-929-dup-0'),
            ('rec-1070-org', 'rec-4833-dup-0'),
            ('rec-1070-org',  'rec-717-dup-0'),
            ('rec-1070-org', 'rec-3984-dup-0'),
            ...
            (  'rec-66-org',  'rec-670-dup-0'),
            (  'rec-66-org', 'rec-4134-dup-0'),
            (  'rec-66-org', 'rec-3866-dup-0'),
            (  'rec-66-org', 'rec-3152-dup-0'),
            (  'rec-66-org', 'rec-3363-dup-0'),
            (  'rec-66-org', 'rec-4495-dup-0'),
            (  'rec-66-org', 'rec-4211-dup-0'),
            (  'rec-66-org', 'rec-3131-dup-0'),
            (  'rec-66-org', 'rec-3815-dup-0'),
            (  'rec-66-org',  'rec-493-dup-0')],
           names=['rec_

In [123]:
indexer = recordlinkage.Index()
indexer.block(left_on='surname', right_on='surname')
pairs = indexer.index(dfA, dfB)
pairs

MultiIndex([('rec-1070-org', 'rec-2672-dup-0'),
            ('rec-1070-org', 'rec-4387-dup-0'),
            ('rec-1070-org',  'rec-787-dup-0'),
            ('rec-1070-org', 'rec-2158-dup-0'),
            ('rec-1016-org', 'rec-2136-dup-0'),
            ('rec-1016-org', 'rec-1948-dup-0'),
            ('rec-1016-org', 'rec-1016-dup-0'),
            ('rec-1016-org', 'rec-1321-dup-0'),
            ('rec-1016-org',  'rec-954-dup-0'),
            ('rec-4405-org', 'rec-2440-dup-0'),
            ...
            ('rec-1003-org', 'rec-3822-dup-0'),
            ('rec-1003-org', 'rec-3462-dup-0'),
            ('rec-1003-org', 'rec-1014-dup-0'),
            ('rec-4883-org', 'rec-1662-dup-0'),
            ('rec-4883-org', 'rec-4883-dup-0'),
            ('rec-4883-org', 'rec-1884-dup-0'),
            ('rec-4883-org',  'rec-785-dup-0'),
            ('rec-4883-org', 'rec-3923-dup-0'),
            ('rec-4883-org', 'rec-2609-dup-0'),
            ('rec-4883-org', 'rec-4459-dup-0')],
           names=['rec_

#### Custom LSH Indexing Algorithm

The PRLT library also allows the Index class to be extended to enable other blocking algorithms. One newer blocking algorithm found in the literature uses Locality Sensitive Hashing (LSH). The LSH algorithm has gained popularity for two reasons. First, it is less strict than standard blocking but still effectively reduces the comparison size relative to full indexing. Second, because the algorithm hashes values before indexing using LSH, it can be used to preserve the privacy of the original data sets (<ins>Dutt, 2023</ins>). In the code below, we extend the PRLT BaseIndexAlgorithm class using the [datasketch library](https://ekzhu.com/datasketch/index.html)'s MinHash and MinHashLSH algorithms. MinHash estimates the Jaccard Similarity Index for two inputs. These are then loaded into the MinHashed LSH index and matches are queried (<ins>Zhu, 2024</ins>). The resulting matches are output as Pandas MultiArray so that they work seamlessly within the PRLT libary.


In [124]:
class LSHIndex(BaseIndexAlgorithm):
    """
    Implements an algorithm for Approximate Nearest Neighbor (ANN) search using
    Locality Sensitive Hashing (LSH). This class is designed to facilitate
    both deduplication within a single dataset and linkage between two datasets
    by finding items with high Jaccard similarity efficiently. It leverages
    MinHash signatures and MinHashLSH for indexed candidate selection.

    :ivar column: Name of the dataframe column containing the data to be used
        for similarity comparison.
    :type column: str
    :ivar threshold: Jaccard similarity threshold between 0 and 1. Only pairs
        above this threshold will be considered as candidates.
    :type threshold: float
    :ivar num_perm: Number of permutations used to create MinHash signatures,
        which directly correlates with the tradeoff between precision and
        recall of the LSH.
    :type num_perm: int
    """

    def __init__(self, column, threshold=0.3, num_perm=128):
        """
        Initializes the LSHIndex class with the specified parameters.

        :param column: Name of the dataframe column to be used for similarity comparison.
        :type column: str
        :param threshold: Jaccard similarity threshold between 0 and 1. Defaults to 0.3.
        :type threshold: float
        :param num_perm: Number of permutations for MinHash signatures. Defaults to 128.
        :type num_perm: int
        """
        super().__init__()
        self.column = column
        self.threshold = threshold
        self.num_perm = num_perm

    def _tokenize(self, string):
        """
        Tokenizes a string into character bigrams after cleaning non-alphanumeric characters.

        :param string: Input string to tokenize.
        :type string: str
        :return: A set of character bigrams extracted from the input string.
        :rtype: set
        """
        # Clean and create character bigrams
        string = re.sub(r'\W+', '', string.lower())
        return set(string[i:i+2] for i in range(len(string)-1))

    def _create_minhash(self, tokens):
        """
        Creates a MinHash object from a set of tokens.

        :param tokens: Set of tokens (strings) to be added to the MinHash object.
        :type tokens: set
        :return: MinHash object representing the token set.
        :rtype: datasketch.MinHash
        """
        m = MinHash(num_perm=self.num_perm)
        for token in tokens:
            m.update(token.encode('utf8'))
        return m

    def _link_index(self, dfA, dfB):
        """
        Links two datasets by creating an LSH index and finding pairs of rows
        with high Jaccard similarity.

        :param dfA: The first dataframe to be linked/deduplicated.
        :type dfA: pandas.DataFrame
        :param dfB: The second dataframe to be linked. Can be the same as dfA for deduplication.
        :type dfB: pandas.DataFrame
        :return: A MultiIndex containing the indices of candidate pairs from the two datasets.
        :rtype: pandas.MultiIndex
        """
        is_deduplication = dfA.equals(dfB)

        # Build MinHash signatures
        minhashes_A = {}
        for idx, row in dfA.iterrows():
            tokens = self._tokenize(row[self.column])
            minhashes_A[idx] = self._create_minhash(tokens)

        if is_deduplication:
            minhashes_B = minhashes_A
        else:
            minhashes_B = {}
            for idx, row in dfB.iterrows():
                tokens = self._tokenize(row[self.column])
                minhashes_B[idx] = self._create_minhash(tokens)

        # Insert into LSH
        lsh = MinHashLSH(threshold=self.threshold, num_perm=self.num_perm)
        for idx, m in minhashes_B.items():
            lsh.insert(str(idx), m)

        # Generate candidate pairs
        pairs = set()
        for idx_a, m_a in minhashes_A.items():
            result = lsh.query(m_a)
            for idx_b in result:
                if is_deduplication and idx_a >= idx_b:
                    continue
                pairs.add((idx_a, idx_b))

        # Return as MultiIndex
        index_array = np.array(list(pairs))
        return pd.MultiIndex.from_arrays([index_array[:, 0], index_array[:, 1]])


#### Indexing Results Using LSHIndex

Below we index our dataset using the custom LSHIndex Blocking algorithm. Using this algorithm to compare the `surname` columns fro the two datasets, we have built a comparison set that is somewhere between full, which we know is an unnecessarily large comparison set, and an exact match blocking set, which is likely too restrictive especially because we have added noise to this column above.

In [125]:
indexer = LSHIndex(column='surname')
pairs_lsh = indexer.index(dfA, dfB)
pairs_lsh

MultiIndex([('rec-3149-org',  'rec-851-dup-0'),
            ( 'rec-356-org', 'rec-2356-dup-0'),
            ('rec-4292-org',   'rec-22-dup-0'),
            ('rec-3623-org', 'rec-4096-dup-0'),
            ('rec-3500-org', 'rec-2415-dup-0'),
            ('rec-2751-org', 'rec-4033-dup-0'),
            ('rec-1352-org', 'rec-4139-dup-0'),
            ('rec-2543-org', 'rec-4071-dup-0'),
            ('rec-4274-org',  'rec-376-dup-0'),
            ('rec-4404-org',  'rec-773-dup-0'),
            ...
            ('rec-2323-org',  'rec-394-dup-0'),
            ('rec-3017-org', 'rec-4582-dup-0'),
            ('rec-4655-org', 'rec-1419-dup-0'),
            ('rec-1271-org', 'rec-4903-dup-0'),
            ('rec-2728-org', 'rec-2334-dup-0'),
            ( 'rec-732-org', 'rec-1295-dup-0'),
            ('rec-4722-org', 'rec-2829-dup-0'),
            ('rec-4662-org', 'rec-4934-dup-0'),
            ('rec-4634-org',  'rec-934-dup-0'),
            ( 'rec-870-org', 'rec-2172-dup-0')],
           names=['rec_

#### Index Runtime Benchmarks:

Time to generate the Pandas MultiIndex for two 5000 row datasets (see Appendix 1 for system info):

- *Full Index Generation*: 118ms / 25M rows
- *Exact-Match Blocking Index Generation*: 25ms / 89727 rows
- *LSH Blocking Index Generation*: 11.6s / 400432 rows


### Comparison

When comparing the candidate records generated by the Indexing steps above in order to generate comparison scores, there are a number of algorithms that can be applied. The PRLT library includes several methods that compare data based on it's type. These comparisons, run alone, all return scores. When a Pandas dataframe is provided with a set of records, the library can be used to compute scores for each row in the dataframe. Each matching column between the two datasets can be assigned it's own comparison algorithm. The result in this case is a vector that includes the row keys of the two rows being compared and columns for each compared variable that includes their match score.

#### Algorithms by Data Type

##### General

  - Exact --> straight equivalence that returns a score of 1 or 0
  - Custom --> As with Indexers above, the library offers the BaseCompareFeature from which custom algorithms can be implemented

##### String

String comparisons are implemented in the PRLT library by wrapping the Python [jellyfish](https://jamesturk.github.io/jellyfish/) library for approximate and phonetic string matching. PRLT implements the following string comparison methods from the jellyfish library:

   - Jaro
   - Jaro-Winkler
   - Levenshtein
   - Damerau-Levenshtein

Interestingly, it excludes Hamming Distance and Match Rating Approach algorithms. The PRLT library also excludes the phonetic encoding tools provided in the jellyfish library. These tools are interesting in that they address phonetic misspellings that might fail edit distance algorithms like the ones above. For example, the spellings Clumps and Klumpz sound the same but would have a low edit distance score. Applying a phonetic encoding tool early can improve match scores (<ins>Zhu, 2024</ins>).

##### Numeric

The PRLT library can compare numeric values using a variety of nearness measures all of which are based on a distance from a comparison point. The following diagram, referenced from the PRLT documentation, well illustrates how the comparisons work (<ins>Elastic, 2025</ins>):

<br><center>
![numeric matching algorithms](images/elas_1705.png)
</center>

As the diagram shows, two values are compared as a distance using one of several decay algorithms (step, linear, exponential, gaussian or squared), returning a float value between 0 and 1.

##### Geospatial

For locations that include lattidue and longitude, the toolkit can compute the haversine distance between coordinates and then compute the numeric similarity between the distances as described for numeric values above.

##### Date

Again, the numeric distance between dates is computed with a few date-specific scoring optimizations applied.

- score when month and day are swapped: allows for a set score to be applied if the only difference between two dates is that day and month are transposed
- score for common month swapping errors: allows for a set score to be applied if the only difference between two dates is that a common string to numeric date conversion error occurred

##### Precomputed Variable

In addition to row-specific values, the toolkit allows for precomputed scores already contained in the dataset to be passed in as variables. By default, these variables are normalized to a score range from 0-1.

#### Comparison Algorithm Applied by Column

- *given_name*: Damerau-Levenshtein distance
- *surname*: Damerau-Levenshtein distance
- *street_number*: Damerau-Levenshtein distance
- *address_1*: Damerau-Levenshtein distance
- *address_2*: Damerau-Levenshtein distance
- *suburb*: Damerau-Levenshtein distance
- *postcode*: Damerau-Levenshtein distance
- *state*: Damerau-Levenshtein distance
- *date_of_birth*: date with default scoring for month/day and text to numeric month errors
- *soc_sec_id*: Damerau-Levenshtein distance

The rationale for using string comparison for almost all fields in this dataset is that most errors detected in a real-world setting are typographical. Thus, using numeric distance scores for street_number, postcode or soc_sec_id are not be appropriate because the underlying error process is not related numeric imprecision or noise. The trade-off is that string-based comparisons are slower than other data-type comparisons with Damerau-Levenshtein being the slowest of the available string algorithms.

In [126]:
comparison = recordlinkage.Compare()
comparison.string(left_on="given_name", right_on="given_name", method='damerau_levenshtein', label='given_name')
comparison.string(left_on="surname", right_on="surname", method='damerau_levenshtein', label='surname')
comparison.string(left_on="street_number", right_on="street_number", method='damerau_levenshtein', label='street_number')
comparison.string(left_on="address_1", right_on="address_1", method='damerau_levenshtein', label='address_1')
comparison.string(left_on="address_2", right_on="address_2", method='damerau_levenshtein', label='address_2')
comparison.string(left_on="suburb", right_on="suburb", method='damerau_levenshtein', label='suburb')
comparison.string(left_on="postcode", right_on="postcode", method='damerau_levenshtein', label='postcode')
comparison.string(left_on="state", right_on="state", method='damerau_levenshtein', label='state')
comparison.string(left_on="soc_sec_id", right_on="soc_sec_id", method='damerau_levenshtein', label='soc_sec_id')
comparison.date(left_on="dob_typed", right_on="dob_typed", label='dob_typed')
print(len(pairs))
features = comparison.compute(pairs=pairs, x=dfA, x_link=dfB)
features

89727


U:\Users\v_mar\OneDrive\Documents\KU\CPSC548\FinalProject\record_linkage\venv\Lib\site-packages\recordlinkage\algorithms\string.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - damerau_levenshtein_distance(x[0], x[1]) / np.max(
U:\Users\v_mar\OneDrive\Documents\KU\CPSC548\FinalProject\record_linkage\venv\Lib\site-packages\recordlinkage\compare.py:414: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  c[


given_name  surname  street_number  address_1  \
rec_id_1     rec_id_2                                                        
rec-1070-org rec-2672-dup-0    0.375000      1.0            0.0   0.642857   
             rec-4387-dup-0    0.250000      1.0            0.0   0.500000   
             rec-787-dup-0     0.125000      1.0            0.0   0.466667   
             rec-2158-dup-0    0.125000      1.0            0.0   0.200000   
rec-1016-org rec-2136-dup-0    0.000000      1.0            0.0   0.235294   
...                                 ...      ...            ...        ...   
rec-4883-org rec-1884-dup-0    0.166667      1.0            0.0   0.166667   
             rec-785-dup-0     0.000000      1.0            0.0   0.181818   
             rec-3923-dup-0    0.000000      1.0            0.0   0.076923   
             rec-2609-dup-0    0.666667      1.0            0.0   0.571429   
             rec-4459-dup-0    0.333333      1.0            0.0   0.615385   

                             address_2    suburb  postcode  state  soc_sec_id  \
rec_id_1     rec_id_2                                                           
rec-1070-org rec-2672-dup-0   0.095238  0.307692      0.00    0.0    0.142857   
             rec-4387-dup-0   0.200000  0.000000      0.00    0.0    0.142857   
             rec-787-dup-0    0.100000  0.076923      0.00    0.0    0.142857   
             rec-2158-dup-0   0.000000  0.307692      0.25    0.0    0.142857   
rec-1016-org rec-2136-dup-0   0.000000  0.111111      0.00    1.0    0.428571   
...                                ...       ...       ...    ...         ...   
rec-4883-org rec-1884-dup-0   0.090909  0.125000      0.25    0.0    0.142857   
             rec-785-dup-0    0.090909  0.111111      0.25    0.0    0.142857   
             rec-3923-dup-0   0.272727  0.111111      0.50    0.0    0.000000   
             rec-2609-dup-0   0.090909  0.222222      0.00    0.0    0.000000   
             rec-4459-dup-0   0.000000  0.181818      0.00    0.0    0.285714   

                             dob_typed  
rec_id_1     rec_id_2                   
rec-1070-org rec-2672-dup-0        0.0  
             rec-4387-dup-0        0.0  
             rec-787-dup-0         0.0  
             rec-2158-dup-0        0.0  
rec-1016-org rec-2136-dup-0        0.0  
...                                ...  
rec-4883-org rec-1884-dup-0        0.0  
             rec-785-dup-0         0.0  
             rec-3923-dup-0        0.0  
             rec-2609-dup-0        0.0  
             rec-4459-dup-0        0.0  

[89727 rows x 10 columns]

In [127]:
features.describe()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,soc_sec_id,dob_typed
count,89727.000000,89727.000000,89727.000000,89727.000000,89727.000000,89727.000000,89727.000000,89727.000000,89727.000000,89727.000000
mean,0.163310,0.945434,0.149767,0.289069,0.139865,0.166778,0.200581,0.284093,0.164450,0.033563
std,0.200799,0.227131,0.265165,0.204610,0.174674,0.185838,0.234500,0.426069,0.199001,0.180094
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.157895,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000
50%,0.142857,1.000000,0.000000,0.235294,0.117647,0.125000,0.250000,0.000000,0.142857,0.000000
75%,0.222222,1.000000,0.333333,0.400000,0.192308,0.200000,0.250000,0.666667,0.285714,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [128]:
features_lsh = comparison.compute(pairs=pairs_lsh, x=dfA, x_link=dfB)
features_lsh

U:\Users\v_mar\OneDrive\Documents\KU\CPSC548\FinalProject\record_linkage\venv\Lib\site-packages\recordlinkage\algorithms\string.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - damerau_levenshtein_distance(x[0], x[1]) / np.max(
U:\Users\v_mar\OneDrive\Documents\KU\CPSC548\FinalProject\record_linkage\venv\Lib\site-packages\recordlinkage\compare.py:388: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  c[


,,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,soc_sec_id,dob_typed
rec_id_1,rec_id_2,,,,,,,,,,
rec-3149-org,rec-851-dup-0,0.000000,0.111111,0.5,0.538462,0.000000,0.000000,0.00,0.000000,0.000000,0.0
rec-356-org,rec-2356-dup-0,0.125000,0.300000,0.0,0.083333,0.000000,0.266667,0.00,0.000000,0.142857,0.0
rec-4292-org,rec-22-dup-0,0.125000,1.000000,0.0,0.066667,0.000000,0.111111,0.50,0.500000,0.142857,0.0
rec-3623-org,rec-4096-dup-0,0.333333,1.000000,0.0,0.153846,0.185185,0.222222,0.25,0.000000,0.000000,0.0
rec-3500-org,rec-2415-dup-0,0.125000,1.000000,0.0,0.222222,0.222222,0.133333,0.00,0.333333,0.142857,0.0
...,...,...,...,...,...,...,...,...,...,...,...
rec-732-org,rec-1295-dup-0,0.000000,0.250000,0.0,0.153846,0.055556,0.285714,0.25,0.000000,0.142857,0.0
rec-4722-org,rec-2829-dup-0,0.111111,0.000000,0.0,0.250000,0.153846,0.214286,0.25,0.000000,0.142857,0.0
rec-4662-org,rec-4934-dup-0,0.285714,0.400000,0.0,0.142857,0.133333,0.133333,0.25,0.000000,0.142857,0.0


In [129]:
features_lsh.describe()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,soc_sec_id,dob_typed
count,400432.000000,400432.000000,400432.000000,400432.000000,400432.000000,400432.000000,400432.000000,400432.000000,400432.000000,400432.000000
mean,0.144277,0.491543,0.131116,0.271939,0.121969,0.145125,0.175834,0.259783,0.141800,0.009447
std,0.159047,0.326849,0.233887,0.177527,0.129006,0.135338,0.198392,0.415547,0.147528,0.096718
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.250000,0.000000,0.153846,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000
50%,0.142857,0.400000,0.000000,0.235294,0.111111,0.125000,0.250000,0.000000,0.142857,0.000000
75%,0.200000,0.800000,0.333333,0.368421,0.181818,0.200000,0.250000,0.333333,0.142857,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Comparison Runtime Benchmarks:

Time to run a comparison of two 5000 row datasets (see Appendix 1 for system info):

- *Full Index*: 27m, 18s
- *Exact-Match Blocking Index*: 5.5s
- *LSH Blocking Index*: 25.1s



## Classification

The PRLT library includes the following classifiers that can be used to classify matches based on the features generated above:

Supervised:

- Logistic Regression Classifier
- Naive Bayesian Classifier
- Support Vector Machine (SVM) Classifier

Unsupervised:

- Expectation/Conditional Maximization (ECM) Classifier
- KMeans Classifier

### Unsupervised Classifiers

In [130]:
# k-means clustering using exact match blocking
kmeans = recordlinkage.KMeansClassifier()
result_kmeans = kmeans.fit_predict(features)
recordlinkage.confusion_matrix(miTrueLinks, result_kmeans)

array([[3367., 1633.],
       [   0.,   nan]])

In [131]:
recordlinkage.fscore(miTrueLinks, result_kmeans)

0.8048284928887295

In [132]:
# k-means clustering using LSH blocking
kmeans = recordlinkage.KMeansClassifier()
result_kmeans = kmeans.fit_predict(features_lsh)
recordlinkage.confusion_matrix(miTrueLinks, result_kmeans)

array([[4231.,  769.],
       [   0.,   nan]])

In [133]:
recordlinkage.fscore(miTrueLinks, result_kmeans)

0.9166937493229335

In [134]:
# ECM using exact blocking
ecm = recordlinkage.ECMClassifier(binarize=0.8)
result_ecm = ecm.fit_predict(features)
recordlinkage.confusion_matrix(miTrueLinks, result_ecm)

array([[3368., 1632.],
       [   0.,   nan]])

In [135]:
recordlinkage.fscore(miTrueLinks, result_ecm)

0.8049713193116634

In [136]:
# ECM using LSH blocking
ecm = recordlinkage.ECMClassifier(binarize=0.8)
result_ecm = ecm.fit_predict(features_lsh)
recordlinkage.confusion_matrix(miTrueLinks, result_ecm)

array([[4233.,  767.],
       [   0.,   nan]])

In [137]:
recordlinkage.fscore(miTrueLinks, result_ecm)

0.9169284089678328

The results above demonstrate that LSH provides better results than exact-match blocking when dealing with dirty data in the blocking set. However, classification performance is not great.

### Supervised Classifiers


In [138]:
# create a training set by splitting data
training_dfA = dfA[0:500]
training_dfB = dfB[0:500]
training_match_idx = miTrueLinks[miTrueLinks.get_level_values(0).isin(training_dfA.index)]

# index training set using LSH
training_pairs_lsh = indexer.index(training_dfA, training_dfB)

# compare training data
training_features_lsh = comparison.compute(pairs=training_pairs_lsh, x=training_dfA, x_link=training_dfB)

#classify using Naive Bayes Classifier
nbc = recordlinkage.NaiveBayesClassifier(binarize=0.8)
nbc.fit(training_features_lsh, training_match_idx)
result_nbc = nbc.predict(features_lsh)
recordlinkage.confusion_matrix(miTrueLinks, result_nbc)


U:\Users\v_mar\OneDrive\Documents\KU\CPSC548\FinalProject\record_linkage\venv\Lib\site-packages\recordlinkage\algorithms\string.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - damerau_levenshtein_distance(x[0], x[1]) / np.max(


array([[4.233e+03, 7.670e+02],
       [2.000e+00,       nan]])

In [139]:
recordlinkage.fscore(miTrueLinks, result_nbc)

0.9167298321602598

In [146]:
#classify using SVM
svc = recordlinkage.SVMClassifier() # options --> ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
svc.fit(training_features_lsh, training_match_idx)
result_svc = svc.predict(features_lsh)
recordlinkage.confusion_matrix(miTrueLinks, result_svc)

array([[4224.,  776.],
       [   0.,   nan]])

In [147]:
recordlinkage.fscore(miTrueLinks, result_svc)

0.9158716392020815

## References

Christen, P. (2019). Data linkage: the big picture. Harvard Data Science Review, 1(2). https://doi.org/10.1162/99608f92.84deb5c4

Christen, P (2012). *Data Matching: Concepts and Techniques for Record Linkage, Entity Resolution, and Duplicate Detection*. Springer. DOI 10.1007/978-3-642-31164-2.

Fellegi, I. P., & Sunter, A. B. (1969). A Theory for Record Linkage. Journal of the American Statistical Association, 64(328), 1183. https://doi.org/10.2307/2286061

Winkler, W. (2002). Methods for Record Linkage and Bayesian Networks Methods for Record Linkage and Bayesian Networks. Retrieved May 5, 2025, from https://www.census.gov/content/dam/Census/library/working-papers/2002/adrm/rrs2002-05.pdf

De Bruin, J. [J535D165]. (2023, July 20). Python Record Linkage Toolkit Documentation — Python Record Linkage Toolkit 0.15 documentation. Retrieved April 17, 2025, from https://recordlinkage.readthedocs.io/en/latest/index.html

Dutt, V. (2023, August 1). Understanding Locality-Sensitive hashing for entity matching. Medium. https://medium.com/@mailvdutt/understanding-locality-sensitive-hashing-for-entity-matching-ebed7998c64b

Zhu, E. (Erik) \[ekzhu\]. (2024, June 3). datasketch: Big Data Looks Small — datasketch 1.6.5 documentation. Retrieved May 6, 2025, from https://ekzhu.com/datasketch/index.html

Turk, J. (2023, November 17). Jellyfish. Retrieved May 6, 2025, from https://jamesturk.github.io/jellyfish/

The closer, the better | ElasticSearch: The Definitive Guide [2.x] | Elastic. (n.d.). Elastic. Retrieved May 6, 2025, from https://www.elastic.co/guide/en/elasticsearch/guide/current/decay-functions.html#img-decay-functions

## Appendix 1: Computing Environment

This is the computing environment used for metrics benchmarks reported in this workbook.

![Computing Environment](images/ReferenceArchitecture.png)

